In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing  import image
import numpy as np
import matplotlib.pyplot as plt
import random

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

x_train = np.repeat(x_train, 3, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)

x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))

x_train = x_train / 255.0
x_test = x_test / 255.0

base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
)

# Freeze base model weights (for transfer learning)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')   # 10 classes for digits 0-9
])

# --- Compile the model ---
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # no one-hot encoding needed
    metrics=['accuracy']
)

# --- Train the model ---
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=3,          # increase epochs for better accuracy
    batch_size=64,
    verbose=1
)

# --- Prediction block ---
p = model.predict(x_test)
r = random.randint(0, 9999)

plt.imshow(x_test[r])
plt.axis('off')
plt.title(f"True Label: {y_test[r]}")
plt.show()

print("Prediction Vector:\n", p[r])
print("Predicted Class Index:", np.argmax(p[r]))